<a href="https://colab.research.google.com/github/GrmRy/Getfix-ML-/blob/main/recomendderGetfix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow import feature_column

from os import getcwd
from sklearn.model_selection import train_test_split

In [ ]:
training=pd.read_csv("/content/TrainingGetfix.csv",delimiter=";")
training.head(20)

,user_id,nama_user,pilihan_layanan,daerah_user,teknisi
0,1,nama1,Teknisi Laptop,Surakarta,1
1,2,nama2,Teknisi Laptop,Surabaya,2
2,3,nama3,Teknisi Laptop,Magelang,3
3,4,nama4,Teknisi Laptop,DKI Jakarta,4
4,5,nama5,Teknisi Handphone,Surakarta,5


In [ ]:
train, test = train_test_split(training, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

112 train examples
28 validation examples
36 test examples


In [ ]:
def df_to_dataset(training, shuffle=True, batch_size=32):
    training = training.copy()
    
    # Use Pandas training's pop method to get the list of targets.
    labels = training.pop('teknisi')# YOUR CODE HERE
    
    # Create a tf.data.Dataset from the training and labels.
    ds = tf.data.Dataset.from_tensor_slices((dict(training), labels.values)) # YOUR CODE HERE
    
    if shuffle:
        # Shuffle dataset.
        ds =ds.shuffle(buffer_size = len(training)) # YOUR CODE HERE
        
    # Batch dataset with specified batch_size parameter.
    ds = ds.batch(batch_size)# YOUR CODE HERE
    
    return ds

In [ ]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of daerah_user:', feature_batch['daerah_user'])
    print('A batch of targets:', label_batch )

Every feature: ['user_id', 'nama_user', 'pilihan_layanan', 'daerah_user']
A batch of ages: tf.Tensor([b'Surabaya' b'Magelang' b'Magelang' b'Magelang' b'Surakarta'], shape=(5,), dtype=string)
A batch of targets: tf.Tensor([ 2  7 15 11  9], shape=(5,), dtype=int64)


In [ ]:
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column, dtype='float64')
    print(feature_layer(example_batch).numpy())

In [ ]:
# EXERCISE: Create a categorical vocabulary column out of the
# above mentioned categories with the key specified as 'thal'.
layanan =feature_column.categorical_column_with_vocabulary_list('pilihan_layanan', ['Teknisi Laptop', 
                                                                                    'Teknisi Handphone', 
                                                                                    'Teknisi Tv',
                                                                                    'Teknisi Komputer']) # YOUR CODE HERE

# EXERCISE: Create an indicator column out of the created categorical column.
thal_one_hot = feature_column.indicator_column(layanan)# YOUR CODE HERE

demo(thal_one_hot)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]


In [ ]:
training.dtypes

user_id             int64
nama_user          object
pilihan_layanan    object
daerah_user        object
teknisi             int64
dtype: object

In [ ]:
# EXERCISE: Fill in the missing code below
feature_columns = []

# Numeric Cols.
# Create a list of numeric columns. Use the following list of columns
numeric_columns = ['user_id']# YOUR CODE HERE

for header in numeric_columns:
    # Create a numeric feature column  out of the header.
    numeric_feature_column =feature_column.numeric_column(header) # YOUR CODE HERE
    
    feature_columns.append(numeric_feature_column)

# Indicator Cols.
# Create a categorical vocabulary column out of the categories
layanan =feature_column.categorical_column_with_vocabulary_list('pilihan_layanan', ['Teknisi Laptop', 
                                                                                    'Teknisi Handphone', 
                                                                                    'Teknisi Tv',
                                                                                    'Teknisi Komputer'])

# Create an indicator column out of the created thal categorical column
layanan_one_hot =feature_column.indicator_column(layanan) # YOUR CODE HERE

feature_columns.append(layanan_one_hot)
daerah=feature_column.categorical_column_with_vocabulary_list('daerah_user',['Surakarta',
                                                                             'Surabaya',
                                                                             'DKI Jakarta',
                                                                             'Magelang'])
daerah_one_hot=feature_column.indicator_column(daerah)
# the dimension parameter.
thal_embedding =feature_column.embedding_column(thal, dimension=8) # YOUR CODE HERE

feature_columns.append(thal_embedding)

# Crossed Cols.
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000) # YOUR CODE HERE

# Create an indicator column out of the crossed column created above to one-hot encode it.
crossed_feature = feature_column.indicator_column(crossed_feature) # YOUR CODE HERE

feature_columns.append(crossed_feature)